In [ ]:
#######################################################
# Script:
#    genDev.py
# Usage:
#    python getnDev.py <input_file> <output_file>
# Description:
#    Generate deviation for last 3 measures
# Authors:
#    Jasmin Nakic, jnakic@salesforce.com
#    Samir Pilipovic, spilipovic@salesforce.com
#######################################################

import math
from datetime import datetime
import numpy as np
from sklearn import linear_model
from sklearn.externals import joblib

# Imports required for visualization (plotly)
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Script debugging flag
debugFlag = False

# Number of valid lines in training data file
limitDev = 1190

In [ ]:
# Iterate over test results
def getStdDev(data):
    X = np.zeros(data.shape[0])
    Y = np.zeros(data.shape[0])
    Z = np.zeros(data.shape[0])
    v = [0,0,0]
    p = [0,0,0]
    idx = 0
    row = 0
    raiseAlert = False
    for m in np.nditer(data):
        idx = idx + 1
        v[0] = v[1] if idx > 2 else 0
        v[1] = v[2] if idx > 1 else 0
        v[2] = m['cnt']
        p[0] = p[1] if idx > 2 else 0
        p[1] = p[2] if idx > 1 else 0
        p[2] = m['predHS']
        if idx > 2:
            # Calculate MA for values and predictions
            X[row] = math.floor((v[0] + v[1] + v[2]) / 3)
            Y[row] = math.floor((p[0] + p[1] + p[2]) / 3)
            # Calculate Deviation
            Z[row] = math.floor(math.sqrt(((p[0] - v[0])**2 + (p[1] - v[1])**2 + (p[2] - v[2])**2)/2))
        row = row + 1
    return (X,Y,Z)
#end getStdDev

In [ ]:
# Write results to file
def writeResult(output,outputTrainDev,calcData,C,P,D):
    # generate result file
    result = np.array(
        np.empty(calcData.shape[0]),
        dtype=[
            ("timeStamp","|U19"),
            ("dateFrac",float),
            ("isHoliday",int),
            ("isSunday",int),
            ("cnt",int),
            ("predSimple",int),
            ("predTrig",int),
            ("predHourDay",int),
            ("predHourWeek",int),
            ("predHS",int),
            ("cntMA",int),
            ("predMA",int),
            ("devMA",int)
        ]
    )

    result["timeStamp"]    = calcData["timeStamp"]
    result["dateFrac"]     = calcData["dateFrac"]
    result["isHoliday"]    = calcData["isHoliday"]
    result["isSunday"]     = calcData["isSunday"]
    result["cnt"]          = calcData["cnt"]
    result["predSimple"]   = calcData["predSimple"]
    result["predTrig"]     = calcData["predTrig"]
    result["predHourDay"]  = calcData["predHourDay"]
    result["predHourWeek"] = calcData["predHourWeek"]
    result["predHS"]       = calcData["predHS"]
    result["cntMA"]        = C
    result["predMA"]       = P
    result["devMA"]        = D
    
    # Last couple measures in input training file are outliers - removing from training file
    resultDev = np.array(
        np.empty(limitDev),
        dtype=[
            ("timeStamp","|U19"),
            ("devMA",int)
        ]
    )
    resultDev['timeStamp'] = result['timeStamp'][0:limitDev]
    resultDev['devMA'] = result['devMA'][0:limitDev]

    if debugFlag:
        print("R 0-5: ", result[0:5])
    hdr = "timeStamp\tdateFrac\tisHoliday\tisSunday\tcnt\tpredSimple\tpredTrig\tpredHourDay\tpredHourWeek\tpredHS\tcntMA\tpredMA\tdevMA"
    np.savetxt(output,result,fmt="%s",delimiter="\t",header=hdr,comments="")
    hdr = "timeStamp\tdevMA"
    np.savetxt(outputTrainDev,resultDev,fmt="%s",delimiter="\t",header=hdr,comments="")
#end writeResult

In [ ]:
# Process deviation for input file
def process(inputFile,outputFile,outputTrainDev):
    # timeStamp dateFrac isHoliday isSunday cnt predSimple predTrig predHourDay predHourWeek predHS
    trainData = np.genfromtxt(
        inputFile,
        delimiter='\t',
        names=True,
        dtype=("|U19",float,int,int,int,int,int,int,int,int)
    )

    (CA,PA,DV) = getStdDev(trainData)
    writeResult(outputFile,outputTrainDev,trainData,CA,PA,DV)
#end process

In [ ]:
# Start - Generate deviation and MA
inputFileName = "train_exc.txt"
outputFileName = "train_ma.txt"
devRawFileName = 'train_dev.txt'

process(inputFileName,devRawFileName,outputTrainDevFile)


# Load results from file generated above using correct data types
results = np.genfromtxt(
    outputFileName,
    dtype=("|U19",float,int,int,int,int,int,int,int,int,int,int,int),
    delimiter='\t',
    names=True
)

# Examine result data
print("Shape:", results.shape)
print("Columns:", len(results.dtype.names), results.dtype.names)
print(results[1:5])
                                            

In [ ]:
# Generate chart with predicitons based on training data (using plotly)
print("Plotly version", __version__) # requires plotly version >= 1.9.0
init_notebook_mode(connected=True)

set1 = go.Bar(
    x=results["dateFrac"],
    y=results["cnt"],
#    marker=dict(color='blue'),
    name='Actual'
)
set2 = go.Bar(
    x=results["dateFrac"],
    y=results["predHS"],
#    marker=dict(color='crimson'),
    opacity=0.6,
    name='Prediction'
)
set3 = go.Bar(
    x=results["dateFrac"],
    y=results["devMA"],
    # marker=dict(color='green'),
    opacity=0.6,
    name='Deviation'
)
barData = [set1, set2, set3]
barLayout = go.Layout(barmode='group', title="Prediction vs. Actual with Deviation")

fig = go.Figure(data=barData, layout=barLayout)
iplot(fig)

In [ ]:
def getOrdinalFrac(dateObj):
    # Return the proleptic Gregorian ordinal of the date + time as date fraction
    dtFrac = dateObj.toordinal() + dateObj.hour/24.0 + dateObj.minute / 1440.0
    return dtFrac
#end getOrdinalFrac

def processLine(data,output):
    # Tab delimited file where col #1 is timestamp and col #2 is the metric value
    vals = data.split('\t')
    timeStamp = vals[0]
    cnt = int(vals[1])
    dt = datetime.strptime(timeStamp,"%Y-%m-%d %H:%M:%S")
    dateStr = dt.strftime("%Y-%m-%d")
    ordinalFrac = getOrdinalFrac(dt)

    dayInWeek = dt.weekday()
    weekdaySin = math.sin(dayInWeek*2*math.pi/7)
    weekdayCos = math.cos(dayInWeek*2*math.pi/7)

    hourInDay = dt.hour
    hourSin = math.sin(hourInDay*2*math.pi/24)
    hourCos = math.cos(hourInDay*2*math.pi/24)

    isMonday = 1    if dayInWeek == 0 else 0
    isTuesday = 1   if dayInWeek == 1 else 0
    isWednesday = 1 if dayInWeek == 2 else 0
    isThursday = 1  if dayInWeek == 3 else 0
    isFriday = 1    if dayInWeek == 4 else 0
    isSaturday = 1  if dayInWeek == 5 else 0
    isSunday = 1    if dayInWeek == 6 else 0

    isHour0 = 1  if hourInDay == 0 else 0
    isHour1 = 1  if hourInDay == 1 else 0
    isHour2 = 1  if hourInDay == 2 else 0
    isHour3 = 1  if hourInDay == 3 else 0
    isHour4 = 1  if hourInDay == 4 else 0
    isHour5 = 1  if hourInDay == 5 else 0
    isHour6 = 1  if hourInDay == 6 else 0
    isHour7 = 1  if hourInDay == 7 else 0
    isHour8 = 1  if hourInDay == 8 else 0
    isHour9 = 1  if hourInDay == 9 else 0
    isHour10 = 1 if hourInDay == 10 else 0
    isHour11 = 1 if hourInDay == 11 else 0
    isHour12 = 1 if hourInDay == 12 else 0
    isHour13 = 1 if hourInDay == 13 else 0
    isHour14 = 1 if hourInDay == 14 else 0
    isHour15 = 1 if hourInDay == 15 else 0
    isHour16 = 1 if hourInDay == 16 else 0
    isHour17 = 1 if hourInDay == 17 else 0
    isHour18 = 1 if hourInDay == 18 else 0
    isHour19 = 1 if hourInDay == 19 else 0
    isHour20 = 1 if hourInDay == 20 else 0
    isHour21 = 1 if hourInDay == 21 else 0
    isHour22 = 1 if hourInDay == 22 else 0
    isHour23 = 1 if hourInDay == 23 else 0

    # Generate input for each hour in a a week
    hourWeek = ""
    for d in range(0,7):
        for h in range(0,24):
            if d > 0 or h > 0:
                hourWeek += "\t"
            if d == dayInWeek and h == hourInDay:
                hourWeek += "1"
            else:
                hourWeek += "0"

    # Holidays in 2016: May 16, Jul 14 and Aug 15
    isHoliday = 0
    if ((dt.month == 5 and dt.day == 16) or
        (dt.month == 7 and dt.day == 14) or
        (dt.month == 8 and dt.day == 15)):
        isHoliday = 1

    # Print the data line
    #      1   2   3   4   5   6     7     8   9     10    11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43
    print("%s\t%s\t%s\t%s\t%s\t%.8f\t%.8f\t%s\t%.8f\t%.8f\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % (timeStamp, dateStr, cnt, ordinalFrac, dayInWeek, weekdaySin, weekdayCos, hourInDay, hourSin, hourCos, isMonday, isTuesday, isWednesday, isThursday, isFriday, isSaturday, isSunday, isHour0, isHour1, isHour2, isHour3, isHour4, isHour5, isHour6, isHour7, isHour8, isHour9, isHour10, isHour11, isHour12, isHour13, isHour14, isHour15, isHour16, isHour17, isHour18, isHour19, isHour20, isHour21, isHour22, isHour23, isHoliday, hourWeek),file=output)
#end processLine

def header(output):
    # Header contains titles for the prediction input data columns
    hourWeekTitle = ""
    for d in range(0,7):
        for h in range(0,24):
            if d > 0 or h > 0:
                hourWeekTitle += "\t"
            hourWeekTitle += "H_" + str(d) + "_" + str(h)
    # Print the header line
    #      1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43
    print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % ("timeStamp", "dateStr", "cnt", "dateFrac", "dayInWeek", "weekdaySin", "weekdayCos", "hourInDay", "hourSin", "hourCos", "isMonday", "isTuesday", "isWednesday", "isThursday", "isFriday", "isSaturday", "isSunday", "isHour0", "isHour1", "isHour2", "isHour3", "isHour4", "isHour5", "isHour6", "isHour7", "isHour8", "isHour9", "isHour10", "isHour11", "isHour12", "isHour13", "isHour14", "isHour15", "isHour16", "isHour17", "isHour18", "isHour19", "isHour20", "isHour21", "isHour22", "isHour23", "isHoliday", hourWeekTitle),file=output)
# end header

In [ ]:
# Start - create Deviation training file with features
linecnt = 0
trainDevFile='train_dev_pred.txt'
f_out = open(trainDevFile,"w") 
with open(devRawFileName) as f_in:
    # Simply read file line by line, skip the header line
    for line in f_in:
        line = line.strip()
        linecnt = linecnt + 1
        header(f_out) if linecnt == 1 else processLine(line,f_out)
f_out.close()

In [ ]:
# Feature list for Deviation Training
devMACols = ["dateFrac", "isHoliday"]
for d in range(0,7):
    for h in range(0,24):
        devMACols.append("H_" + str(d) + "_" + str(h))

hourHolidayCols  = ["isHoliday",
    "isHour0", "isHour1", "isHour2", "isHour3", "isHour4", "isHour5", "isHour6", "isHour7",
    "isHour8", "isHour9", "isHour10", "isHour11", "isHour12", "isHour13", "isHour14", "isHour15",
    "isHour16", "isHour17", "isHour18", "isHour19", "isHour20", "isHour21", "isHour22", "isHour23"]
hourSundayCols  = ["isSunday",
    "isHour0", "isHour1", "isHour2", "isHour3", "isHour4", "isHour5", "isHour6", "isHour7",
    "isHour8", "isHour9", "isHour10", "isHour11", "isHour12", "isHour13", "isHour14", "isHour15",
    "isHour16", "isHour17", "isHour18", "isHour19", "isHour20", "isHour21", "isHour22", "isHour23"]

In [ ]:
# Add columns to the data set
def addColumns(dest, src, colNames):
    # Initialize temporary array
    tmpArr = np.empty(src.shape[0])
    cols = 0
    # Copy column content
    for name in colNames:
        if cols == 0: # first column
            tmpArr = np.copy(src[name])
            tmpArr = np.reshape(tmpArr,(-1,1))
        else:
            tmpCol = np.copy(src[name])
            tmpCol = np.reshape(tmpCol,(-1,1))
            tmpArr = np.append(tmpArr,tmpCol,1)
        cols = cols + 1
    return np.append(dest,tmpArr,1)
#end addColumns

# Generate model for deviation MA
def genModel(data,colList,modelName):
    # Initialize array
    X = np.zeros(data.shape[0])
    X = np.reshape(X,(-1,1))

    # Add columns
    X = addColumns(X,data,colList)

    if debugFlag:
        print("X 0: ", X[0:5])

    Y = np.copy(data["cnt"])
    if debugFlag:
        print("Y 0: ", Y[0:5])

    model = linear_model.LinearRegression()
    print(model.fit(X, Y))

    print("INTERCEPT: ", model.intercept_)
    print("COEFFICIENT shape: ", model.coef_.shape)
    print("COEFFICIENT values: ", model.coef_)
    print("SCORE values: ", model.score(X,Y))

    P = model.predict(X)
    if debugFlag:
        print("P 0-5: ", P[0:5])

    joblib.dump(model,modelName)
    return P
#end genModel

In [ ]:
# Write MA
def writeResultDev(output,data,p):
    # generate result file
    result = np.array(
       np.empty(data.shape[0]),
       dtype=[
           ("timeStamp","|U19"),
           ("dateFrac",float),
           ("cnt",int),
           ("predDev",int)
        ]
    )

    result["timeStamp"]    = data["timeStamp"]
    result["dateFrac"]     = data["dateFrac"]
    result["cnt"]          = data["cnt"]
    result["predDev"]      = p

    if debugFlag:
        print("R 0-5: ", result[0:5])
    hdr = "timeStamp\tdateFrac\tcntDev\tpredDev"
    np.savetxt(output,result,fmt="%s",delimiter="\t",header=hdr,comments="")
#end writeResultDev

In [ ]:
# Start
trainPass1DevMA = "train_devMA.txt"

# All input columns - data types are strings, float and int
trainData = np.genfromtxt(
    trainDevFile,
    delimiter='\t',
    names=True,
    dtype=("|U19","|S10",int,float,int,float,float,int,float,float,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int,
           int,int,int,int,int,int,int,int,int,int
    )
)

P1 = genModel(trainData,devMACols,"modelTrainDev")

writeResultDev(trainPass1DevMA,trainData,P1)

In [ ]:
# Load the training data from file generated above using correct data types# Load  
resultsDevMA = np.genfromtxt(
    trainPass1DevMA,
    dtype=("|U19",float,int,int),
    delimiter='\t',
    names=True
)

# Examine training data
print("Shape:", resultsDevMA.shape)
print("Columns:", len(resultsDevMA.dtype.names), resultsDevMA.dtype.names)
print(resultsDevMA[1:5])

In [ ]:
# Generate chart with predicitons based on training data (using plotly)
print("Plotly version", __version__) # requires plotly version >= 1.9.0
init_notebook_mode(connected=True)

set1 = go.Bar(
    x=resultsDevMA["dateFrac"],
    y=resultsDevMA["cntDev"],
#    marker=dict(color='blue'),
    name='Actual'
)
set2 = go.Bar(
    x=resultsDevMA["dateFrac"],
    y=resultsDevMA["predDev"],
#    marker=dict(color='crimson'),
    opacity=0.6,
    name='Prediction'
)
barData = [set1, set2]
barLayout = go.Layout(barmode='group', title="Prediction vs. Actual")

fig = go.Figure(data=barData, layout=barLayout)
iplot(fig)

In [ ]:
# Write results to file
def writeResultDetect(output,predData,P):
    # generate result file
    result = np.array(
        np.empty(predData.shape[0]),
        dtype=[
            ("timeStamp","|U19"),
            ("dateFrac",float),
            ("isHoliday",int),
            ("isSunday",int),
            ("cnt",int),
            ("predSimple",int),
            ("predTrig",int),
            ("predHourDay",int),
            ("predHourWeek",int),
            ("predHS",int),
            ("cntMA",int),
            ("predMA",int),
            ("devMA",int),
            ("predDev",int)
        ]
    )

    result["timeStamp"]    = predData["timeStamp"]
    result["dateFrac"]     = predData["dateFrac"]
    result["isHoliday"]    = predData["isHoliday"]
    result["isSunday"]     = predData["isSunday"]
    result["cnt"]          = predData["cnt"]
    result["predSimple"]   = predData["predSimple"]
    result["predTrig"]     = predData["predTrig"]
    result["predHourDay"]  = predData["predHourDay"]
    result["predHourWeek"] = predData["predHourWeek"]
    result["predHS"]       = predData["predHS"]
    result["cntMA"]        = predData["cntMA"]
    result["predMA"]       = predData["predMA"]
    result["devMA"]        = predData["devMA"]
    result["predDev"]      = P

    if debugFlag:
        print("R 0-5: ", result[0:5])
    hdr = "timeStamp\tdateFrac\tisHoliday\tisSunday\tcnt\tpredSimple\tpredTrig\tpredHourDay\tpredHourWeek\tpredHS\tcntMA\tpredMA\tdevMA\tpredDev"
    np.savetxt(output,result,fmt="%s",delimiter="\t",header=hdr,comments="")
#end writeResultDetect

In [ ]:
# Start
trainAnomFile = "train_anom.txt"
writeResultDetect(trainAnomFile,results[0:limitDev],P)